In [3]:
import torch
import os
from dotenv import load_dotenv
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonAstREPLTool
from langchain.text_splitter import RecursiveCharacterTextSplitter
import tiktoken
from langchain.embeddings import OpenAIEmbeddings
import pinecone
from langchain.vectorstores import Pinecone

In [5]:
load_dotenv('./.env')
os.environ.get('PINECONE_ENV')

'gcp-starter'

In [6]:
device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'
num_gpus = torch.cuda.device_count()

if num_gpus > 0:
    print(f"GPUs available: {num_gpus}")
    for i in range(num_gpus):
        gpu_name = torch.cuda.get_device_name(i)
        print(f"GPU {i}: {gpu_name}")
else:
    print("There is no GPU available. Using CPU")

GPUs available: 1
GPU 0: NVIDIA RTX A4000


### ChatModels: GPT-3.5-Turbo e GPT-4

In [7]:
chat = ChatOpenAI(
    model_name='gpt-3.5-turbo',
    temperature=0.5,
    max_tokens=1024
)

In [8]:
messages = [
    SystemMessage(content="Voce e um especialista em machine learning que responde tudo em portugues do Brasil"),
    HumanMessage(content="explique em um paragrafo o que e machine learning.")
]

In [9]:
output = chat(messages)

In [10]:
print(output.content)

Machine learning é uma área da inteligência artificial que se baseia em algoritmos e modelos estatísticos para permitir que um sistema aprenda a partir de dados, sem ser explicitamente programado. Essa abordagem permite que as máquinas adquiram conhecimento e habilidades a partir da análise de grandes quantidades de dados, possibilitando a tomada de decisões, previsões e resolução de problemas de forma automatizada e mais eficiente.


### Prompt Templates

In [35]:
meu_template = """Você é um virologista experiente.
Escreva 5 frases sobre o vírus {virus} no idioma {idioma}."""

prompt = PromptTemplate(
    input_variables=['virus', 'idioma'],
    template=meu_template
)

print(prompt)

input_variables=['idioma', 'virus'] template='Você é um virologista experiente.\nEscreva 5 frases sobre o vírus {virus} no idioma {idioma}.'


In [38]:
llm = OpenAI(
    model_name='text-davinci-003',
    temperature=0.7
)

output = llm(prompt.format(virus='covid-19', idioma='portugues do brasil'))

In [39]:
print(output)



1. O vírus covid-19 pode ser transmitido de pessoa para pessoa através de contato próximo ou por gotículas de saliva. 
2. É importante praticar medidas de higiene frequentes, como lavar as mãos ou usar álcool em gel, para evitar a propagação do vírus.
3. O vírus covid-19 pode causar sintomas leves a graves, como tosse, febre, falta de ar e dificuldades respiratórias.
4. As pessoas de maior risco para a doença são aquelas com idade avançada ou que possuem doenças preexistentes.
5. A melhor maneira de prevenir a propagação do vírus covid-19 é evitando contato próximo com outras pessoas e seguindo as orientações das autoridades de saúde.


### Simple Chains

In [41]:
llm = ChatOpenAI(
    model_name='gpt-4',
    temperature=0.5
)

meu_template = """Você é um virologista experiente.
Escreva um resumo sobre o vírus {virus} no idioma {idioma}."""

prompt = PromptTemplate(
    input_variables=['virus', 'idioma'],
    template=meu_template
)

chain = LLMChain(llm=llm, prompt=prompt)

In [42]:
output = chain.run({
    'virus':'covid-19','idioma':'portugues do brasil'
})

In [43]:
print(output)

O COVID-19 é uma doença infecciosa causada pelo coronavírus SARS-CoV-2. Foi identificado pela primeira vez em Wuhan, China, em dezembro de 2019 e, desde então, desencadeou uma pandemia global. O vírus se espalha principalmente através de gotículas produzidas quando uma pessoa infectada tosse, espirra ou fala. Também pode se espalhar tocando uma superfície ou objeto contaminado e depois tocando a boca, nariz ou olhos.

Os sintomas do COVID-19 variam em gravidade, desde infecções assintomáticas ou leves a casos graves ou críticos. Os sintomas mais comuns incluem febre, tosse seca e cansaço, mas também podem incluir dores no corpo, dor de garganta, perda de paladar ou olfato, e dificuldade para respirar. Em casos graves, pode levar a pneumonia, síndrome respiratória aguda grave e morte.

A prevenção da COVID-19 envolve medidas de saúde pública para reduzir a transmissão do vírus. Isso inclui distanciamento físico, uso de máscaras faciais, higienização frequente das mãos, cobrir a boca ao 

### Sequencial Chains

In [18]:
#--- primeira chain ---

llm1 = OpenAI(
    model_name='text-davinci-003',
    temperature=0.7,
    max_tokens=1024
)

prompt1 = PromptTemplate(
    input_variables=['conceito'],
    template="""
    Voce e um cientista experiente e programador Python.
    Escreva uma funcao que implementa o conceito de {conceito}"""
)

chain1 = LLMChain(llm=llm1, prompt=prompt1)

In [26]:
# --- segunda chain ---

llm2 = ChatOpenAI(
    model_name='gpt-3.5-turbo',
    temperature=1.2
)

prompt2 = PromptTemplate(
    input_variables=['function'],
    template="""Dada a funcao Python {function}, descreva como funciona da forma mais detalhada possivel e por ultimo exiba a o codigo da funcao"""
)

chain2 = LLMChain(llm=llm2, prompt=prompt2)

In [27]:
overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)
output = overall_chain.run('regressao linear')



> Entering new SimpleSequentialChain chain...
.

def linear_regression(x, y):
    '''Esta funcao implementa o conceito de regressao linear.
    
    Ela calcula a linha de regressao que melhor se ajusta aos dados x e y,
    usando o metodo dos minimos quadrados.
    
    Args:
        x: Lista de valores de x.
        y: Lista de valores de y.
        
    Retorna:
        A inclinacao (m) e o intercepto (b) da linha de regressao.
    '''
    N = len(x)
    
    # Calcula a media dos valores de x e y
    x_mean = sum(x)/N
    y_mean = sum(y)/N
    
    # Calcula os coeficientes da regressao linear
    numerador = 0
    denominador = 0
    for i in range(N):
        numerador += (x[i] - x_mean) * (y[i] - y_mean)
        denominador += (x[i] - x_mean)**2
    m = numerador/denominador
    b = y_mean - m*x_mean
    
    return m, b
A função `linear_regression` implementa o conceito de regressão linear, que é um método estatístico para encontrar a melhor linha de ajuste aos dados.
A funçã

In [28]:
print(output)

A função `linear_regression` implementa o conceito de regressão linear, que é um método estatístico para encontrar a melhor linha de ajuste aos dados.
A função recebe duas listas de valores, `x` e `y`, que representam os pontos no gráfico. Ela calcula a inclinação (m) e o intercepto (b) da linha de regressão que melhor se ajusta aos dados usando o método dos mínimos quadrados.

A função começa obtendo o número de elementos nas listas `x` e `y`. Isso é armazenado em `N` por meio da função `len()`.

Em seguida, a função calcula a média dos valores de `x` e `y`. Isso é feito somando todos os elementos da lista e dividindo pelo total de elementos. A média de `x` é armazenada em `x_mean`, e a média de `y` é armazenada em `y_mean`.

Agora a função inicia o cálculo dos coeficientes da regressão linear. Ela usa um loop `for` para percorrer todos os elementos das listas `x` e `y`. Para cada elemento, ela calcula as diferenças em relação às médias de `x` e `y` e as multiplica. Esses valores são 

### LangChain Agents

In [7]:
llm = OpenAI(temperature=0)
agent_executor = create_python_agent(
    llm=llm,
    tool=PythonAstREPLTool(),
    verbose=True
)

In [5]:
agent_executor.run('calcule a raiz quadrada do fatorial de 20 e exiba com quatro casas decimais')



> Entering new AgentExecutor chain...
 I need to calculate the square root of the factorial of 20
Action: python_repl_ast
Action Input: from math import factorial; print(f"{round(factorial(20)**0.5, 4)}")
Observation: 1559776268.6285

Thought: I now know the final answer
Final Answer: 1559776268.6285

> Finished chain.


'1559776268.6285'

### Splitting e Embedding de texto

In [127]:
with open('docs/nat.txt') as f:
    clt = f.read()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=10,
    length_function=len
)

In [128]:
chunks = text_splitter.create_documents([clt])

In [129]:
len(chunks)


9

In [146]:
print(chunks[0].page_content)

NAT: Núcleo de Apoio Técnico

Coordenadora-Geral: Marcela Cristina Ozório
Coordenador-Adjunto: Bernardo Fiterman Albano


In [147]:
def embedding_cost(texts):
    enc = tiktoken.encoding_for_model('text-embedding-ada-002')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    total_em_dolar = total_tokens / 1000 * 0.0001
    print(f'Total de tokens: {total_tokens}')
    print(f'Custo de Embedding em USD: {total_em_dolar:.6f}')
    print(f'Custo de Embedding em R$: {total_em_dolar * 4.95:.6f}')

In [148]:
embedding_cost(chunks)

Total de tokens: 818
Custo de Embedding em USD: 0.000082
Custo de Embedding em R$: 0.000405


In [149]:
embeddings = OpenAIEmbeddings()

In [150]:
vector = embeddings.embed_query(chunks[0].page_content)

In [151]:
print(vector)

[-0.024427912094871006, 0.00519900227828066, 0.019773696288307465, -0.020769106928320225, -0.011110933874245265, 0.030238959100842643, -0.00589511703197114, -0.011339608964324037, 0.004334743747300751, 0.004089254093228515, 0.011938200736968595, -0.015146381735223736, 0.002322064092877294, -0.011252174535409672, -0.0007280620876736171, 0.004906432403820288, 0.010014636781594326, -0.014581419090566107, -0.007687528878312557, -0.014621774230414941, -0.00527634862072477, 0.01502531910964505, 0.01261750092666561, 0.0011349697982116897, 0.012711661367441882, -0.011709525646889823, -0.0023506484791371404, -0.03900933397940493, 0.018509256349689706, -0.04315239337356088, 0.014998415993520028, -0.0019252451090937671, 0.020029273563188652, -0.03548504159951144, -0.010949516108817742, -0.020083079795438698, 0.00032472748362568813, -0.016209049699887745, 0.013626363590402183, -0.0215223904545304, 0.010384553464160113, 0.005568918495185143, 0.010707388995015157, -0.016948883065019318, -0.010283667

## Vector

In [152]:
pinecone.init(api_key=os.environ.get('PINECONE_API_KEY'), environment=os.environ.get('PINECONE_ENV'))

In [153]:
indexes = pinecone.list_indexes()
for i in indexes:
    print('Index encontrada e apagada: ' + i)
    pinecone.delete_index(i)

Index encontrada e apagada: linuxtips


In [154]:
index_name = 'linuxtips'
if index_name not in pinecone.list_indexes():
    pinecone.create_index(index_name, dimension=1536, metric='cosine')
    print('Index '+index_name+' criado')

Index linuxtips criado


In [155]:
vector_store = Pinecone.from_documents(chunks, embeddings, index_name=index_name)

### Conversando com dados

In [157]:
query = 'site do nat'
result = vector_store.similarity_search(query)
print(result)

[Document(page_content='O acesso para registro e acompanhamento de pedidos de apoio ao NAT pode\nser feito por meio do endereço: https://sigep.mpac.mp.br\n\n\nSite do NAT: https://nat.mpac.mp.br'), Document(page_content='Prova disso é que no ano de 2019, o NAT atendeu diversas demandas de\nbaixa, média e alta complexidade, em processos e/ou procedimentos judiciais e\nextrajudiciais, além de dar suporte ao Grupo de Atuação Especial de Combate ao\nCrime Organizado – GAECO na condução de operações importantes e de suas\natividades diárias.'), Document(page_content='O Núcleo de Apoio Técnico (NAT) foi instituído no âmbito do Ministério\nPúblico do Estado do Acre por meio do Ato n.º 25, de 13 de setembro de 2012, da\nProcuradoria-Geral de Justiça, com o escopo de prestar apoio de inteligência e\nsegurança institucional, técnico-científico e operacional, por meio de servidores\nhabilitados em áreas de conhecimento específico, aos órgãos de execução e, em\nespecial, ao Grupo de Atuação Especi

In [158]:
for r in result:
    print(r.page_content)
    print('-' * 50)

O acesso para registro e acompanhamento de pedidos de apoio ao NAT pode
ser feito por meio do endereço: https://sigep.mpac.mp.br


Site do NAT: https://nat.mpac.mp.br
--------------------------------------------------
Prova disso é que no ano de 2019, o NAT atendeu diversas demandas de
baixa, média e alta complexidade, em processos e/ou procedimentos judiciais e
extrajudiciais, além de dar suporte ao Grupo de Atuação Especial de Combate ao
Crime Organizado – GAECO na condução de operações importantes e de suas
atividades diárias.
--------------------------------------------------
O Núcleo de Apoio Técnico (NAT) foi instituído no âmbito do Ministério
Público do Estado do Acre por meio do Ato n.º 25, de 13 de setembro de 2012, da
Procuradoria-Geral de Justiça, com o escopo de prestar apoio de inteligência e
segurança institucional, técnico-científico e operacional, por meio de servidores
habilitados em áreas de conhecimento específico, aos órgãos de execução e, em
especial, ao Grupo de A

### Conversando com LLM sobre dados

In [159]:
llm = ChatOpenAI(model='gpt-3.5-turbo-16k', temperature=0.5)
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k':3})
chain = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=retriever)

In [166]:
query = 'me da um resumo sobre o que é o nat'
resp = chain.run(query)

In [167]:
print(resp)

O Núcleo de Apoio Técnico (NAT) é um órgão do Ministério Público do Estado do Acre que foi instituído com o objetivo de fornecer apoio técnico, científico e operacional aos membros e órgãos do Ministério Público, em especial ao Grupo de Atuação Especial no Combate ao Crime Organizado (GAECO). O NAT atende demandas de baixa, média e alta complexidade, tanto em processos judiciais como extrajudiciais, e também dá suporte ao GAECO em suas atividades diárias e na condução de operações importantes. Desde a sua criação, o NAT tem sido fundamental para o desenvolvimento das atividades finais do Ministério Público.
